# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

In [2]:
tf.__version__

'2.5.0'

### Definitions

In [3]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr_tcn"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow_lite_on_aws_lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 318


### helper functions

In [4]:
# save file before update
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))

### load data

In [5]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,extrap,pos,age_pos,test,age_test,day_num,nb_cases,sum_cases,Rt,rate_pos,train
date,,,,,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,0.0,881,61.104427,39013,55.451567,3,141108,NaN,NaN,2.258222,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,0.0,981,60.403670,41975,54.832186,4,142089,NaN,NaN,2.337105,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,0.0,1019,60.063788,47596,54.404446,5,143108,NaN,NaN,2.140936,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,0.0,291,60.020619,16523,54.480058,6,143399,NaN,NaN,1.761181,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,0.0,141,61.248227,6611,58.226895,0,143540,NaN,NaN,2.132809,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-10,289.186667,2021-06-10,297.723333,54.983333,86.416667,0.0,3787,38.742804,336583,38.754007,4,5669834,84142.0,0.543574,1.125131,False
2021-06-11,289.486667,2021-06-11,297.756667,55.766667,87.200000,0.0,3923,38.684935,376755,40.348685,5,5673757,77487.0,0.523578,1.041260,False
2021-06-12,289.910000,2021-06-12,298.000000,56.283333,87.433333,0.0,2170,38.973272,195235,45.751515,6,5675927,73344.0,0.504349,1.111481,False


### prepare features

In [6]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,881,39013,3,61.104427,55.451567
2020-05-14,285.050000,290.963333,59.406780,84.847458,981,41975,4,60.403670,54.832186
2020-05-15,285.308333,291.920000,57.372881,82.966102,1019,47596,5,60.063788,54.404446
2020-05-16,284.956667,293.500000,53.741379,86.534483,291,16523,6,60.020619,54.480058
2020-05-17,285.598333,294.446667,49.879310,85.500000,141,6611,0,61.248227,58.226895
...,...,...,...,...,...,...,...,...,...
2021-06-10,289.186667,297.723333,54.983333,86.416667,3787,336583,4,38.742804,38.754007
2021-06-11,289.486667,297.756667,55.766667,87.200000,3923,376755,5,38.684935,40.348685
2021-06-12,289.910000,298.000000,56.283333,87.433333,2170,195235,6,38.973272,45.751515


### Prepare dataset

In [7]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [8]:
dataset.shape[1]

9

In [9]:
PAST_HISTORY

14

In [10]:
dataset.shape[1]

9

### Load model

In [11]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)

/anaconda3/envs/coronavirusModel/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1061: UserWarning: tcn.tcn is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


CPU times: user 1.19 s, sys: 79.3 ms, total: 1.27 s
Wall time: 1.39 s


In [12]:
multi_step_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14, 9)]           0         
_________________________________________________________________
tcn (TCN)                    (None, 64)                43136     
_________________________________________________________________
dense (Dense)                (None, 7)                 455       
_________________________________________________________________
dropout (Dropout)            (None, 7)                 0         
Total params: 43,591
Trainable params: 43,591
Non-trainable params: 0
_________________________________________________________________


In [13]:
multi_step_model.inputs[0].dtype

tf.float32

In [14]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_tcn"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: ./keras_tcn/assets


INFO:tensorflow:Assets written to: ./keras_tcn/assets


### Save model TFlite

In [15]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow_lite_on_aws_lambda/converted_model_20210622_08_25_36.tflite moved!


In [16]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

In [17]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

188892

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [18]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[321 - 335]
[328 - 342]
[335 - 349]
[342 - 356]
[349 - 363]
[356 - 370]
[363 - 377]
[370 - 384]
[377 - 391]
9


array([[29666.922  , 26996.371  , 28165.648  , 31385.72   , 19210.123  ,
         9948.079  , 34728.113  , 28436.117  , 27424.426  , 24031.824  ,
        26893.992  , 19191.252  , 10096.86   , 30991.406  , 24586.94   ,
        23193.324  , 20728.414  , 24644.582  , 21748.125  , 10572.421  ,
        27509.447  , 20609.107  , 21189.688  , 17618.365  , 18332.193  ,
        17891.316  , 13277.501  , 29824.629  , 17055.047  , 14979.952  ,
        13569.202  , 17585.182  , 10375.552  ,  6525.003  , 25564.52   ,
        17005.709  , 14123.661  ,  9407.52   , 15494.065  , 11914.357  ,
         9765.455  , 22527.896  , 13434.8    , 14472.842  , 12226.064  ,
        12567.557  , 10380.274  ,  8974.154  ,  9123.854  , 10548.352  ,
        11262.53   ,  9826.993  ,  9020.069  ,  4786.5684 ,  1300.3066 ,
        15870.695  , 10750.324  , 10246.054  ,  4879.8633 ,  7482.1123 ,
         2868.712  ,   512.15234, 13497.69   ]], dtype=float32)

#### Future days

In [19]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [20]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [21]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [22]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

### Predict reloaded model

#### Past days

In [23]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [24]:
x_multi.shape

(1, 14, 9)

In [25]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [26]:
dataset.shape

(398, 9)

In [27]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[321 - 335]
[328 - 342]
[335 - 349]
[342 - 356]
[349 - 363]
[356 - 370]
[363 - 377]
[370 - 384]
[377 - 391]


In [28]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [29]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [30]:
y_multi_pred

array([[ 1.1958351 ,  1.0042262 ,  1.0881205 ,  1.3191568 ,  0.4455724 ,
        -0.218968  ,  1.5589695 ,  1.1075264 ,  1.0349387 ,  0.7915236 ,
         0.99688077,  0.4442185 , -0.20829311,  1.2908653 ,  0.83135253,
         0.7313622 ,  0.5545079 ,  0.8354882 ,  0.62767106, -0.1741722 ,
         1.0410389 ,  0.54594785,  0.5876038 ,  0.3313658 ,  0.38258207,
         0.3509497 ,  0.01991402,  1.2071503 ,  0.29094827,  0.14206283,
         0.0408432 ,  0.32898492, -0.18829736, -0.46456954,  0.9014928 ,
         0.28740835,  0.08062498, -0.25775248,  0.1789498 , -0.0778899 ,
        -0.23207107,  0.68361866,  0.0312    ,  0.10567823, -0.05552532,
        -0.03102365, -0.18795848, -0.28884587, -0.27810514, -0.1758992 ,
        -0.12465773, -0.22765577, -0.28555155, -0.58930016, -0.83943516,
         0.2059726 , -0.16140789, -0.19758867, -0.5826063 , -0.3958981 ,
        -0.726904  , -0.8959843 ,  0.03571232]], dtype=float32)

In [31]:
y_multi_pred_out

array([[ 1.19583523,  1.00422609,  1.08812034,  1.319157  ,  0.44557238,
        -0.21896777,  1.55896986,  1.1075263 ,  1.03493869,  0.79152352,
         0.99688053,  0.44421849, -0.2082932 ,  1.2908653 ,  0.83135241,
         0.73136199,  0.55450785,  0.8354882 ,  0.62767106, -0.17417222,
         1.04103875,  0.54594767,  0.58760369,  0.33136582,  0.38258219,
         0.3509497 ,  0.01991422,  1.20715046,  0.29094827,  0.14206269,
         0.0408432 ,  0.3289848 , -0.18829742, -0.46456942,  0.90149271,
         0.28740838,  0.08062491, -0.25775248,  0.17894992, -0.07788995,
        -0.2320711 ,  0.68361866,  0.03119985,  0.10567832, -0.05552541,
        -0.03102371, -0.18795848, -0.28884581, -0.27810514, -0.1758991 ,
        -0.1246579 , -0.22765586, -0.28555155, -0.58930022, -0.83943516,
         0.20597251, -0.16140766, -0.19758859, -0.58260596, -0.39589795,
        -0.7269038 , -0.89598417,  0.03571235]])

In [32]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [33]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [34]:
y_future_pred_out

array([[-0.5843907 , -0.3629649 , -0.48085707, -0.53998703, -0.89822507,
        -0.9459039 , -0.35014218]])

In [35]:
y_future_pred

array([[-0.5843906 , -0.36296496, -0.4808569 , -0.5399871 , -0.8982251 ,
        -0.9459039 , -0.35014212]], dtype=float32)

In [36]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [37]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow_lite_on_aws_lambda\nserverless deploy -v'

In [38]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [39]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow_lite_on_aws_lambda
serverless deploy -v

In [40]:
!./deploy_serverless.sh

Serverless: Deprecation warning: Resolution of lambda version hashes was improved with better algorithm, which will be used in next major release.
            Switch to it now by setting "provider.lambdaHashingVersion" to "20201221"
            More Info: https://www.serverless.com/framework/docs/deprecations/#LAMBDA_HASHING_VERSION_V2
Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/eddadf68f9f2c21cf494701082a5393692d7b3c4711137913b3ed1b03bec68cf_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: In

### API AWS real Test

#### Past days

In [41]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[321 - 335]
[328 - 342]
[335 - 349]
[342 - 356]
[349 - 363]
[356 - 370]
[363 - 377]
[370 - 384]
[377 - 391]
status code :  200
[[[1.1958352327346802, 1.0042260885238647, 1.0881203413009644, 1.3191570043563843, 0.44557255506515503, -0.21896782517433167, 1.5589698553085327]], [[1.1075263023376465, 1.0349386930465698, 0.7915234565734863, 0.9968806505203247, 0.4442184567451477, -0.20829322934150696, 1.2908653020858765]], [[0.8313524127006531, 0.7313621044158936, 0.5545077919960022, 0.8354882001876831, 0.6276711225509644, -0.17417222261428833, 1.0410387516021729]], [[0.5459477305412292, 0.5876038074493408, 0.3313658833503723, 0.3825821876525879, 0.3509496748447418, 0.019914261996746063, 1.2071503400802612]], [[0.2909482717514038, 0.14206287264823914, 0.04084325581789017, 0.3289848566055298, -0.18829742074012756, -0.4645695388317108, 0.9014928936958313]], [[0.28740838170051575, 0.0806250274181366, -0.2577524781227112, 0.17894989252090454, -0.07788991928100586, -0.23207107186317444, 0.6836186

In [42]:
len(json_list_list_x)

23817

In [43]:
resp.json()

[[[1.1958352327346802,
   1.0042260885238647,
   1.0881203413009644,
   1.3191570043563843,
   0.44557255506515503,
   -0.21896782517433167,
   1.5589698553085327]],
 [[1.1075263023376465,
   1.0349386930465698,
   0.7915234565734863,
   0.9968806505203247,
   0.4442184567451477,
   -0.20829322934150696,
   1.2908653020858765]],
 [[0.8313524127006531,
   0.7313621044158936,
   0.5545077919960022,
   0.8354882001876831,
   0.6276711225509644,
   -0.17417222261428833,
   1.0410387516021729]],
 [[0.5459477305412292,
   0.5876038074493408,
   0.3313658833503723,
   0.3825821876525879,
   0.3509496748447418,
   0.019914261996746063,
   1.2071503400802612]],
 [[0.2909482717514038,
   0.14206287264823914,
   0.04084325581789017,
   0.3289848566055298,
   -0.18829742074012756,
   -0.4645695388317108,
   0.9014928936958313]],
 [[0.28740838170051575,
   0.0806250274181366,
   -0.2577524781227112,
   0.17894989252090454,
   -0.07788991928100586,
   -0.23207107186317444,
   0.6836186647415161]],
 

In [44]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [45]:
y_multi_pred_out

array([[ 1.19583523,  1.00422609,  1.08812034,  1.319157  ,  0.44557256,
        -0.21896783,  1.55896986,  1.1075263 ,  1.03493869,  0.79152346,
         0.99688065,  0.44421846, -0.20829323,  1.2908653 ,  0.83135241,
         0.7313621 ,  0.55450779,  0.8354882 ,  0.62767112, -0.17417222,
         1.04103875,  0.54594773,  0.58760381,  0.33136588,  0.38258219,
         0.35094967,  0.01991426,  1.20715034,  0.29094827,  0.14206287,
         0.04084326,  0.32898486, -0.18829742, -0.46456954,  0.90149289,
         0.28740838,  0.08062503, -0.25775248,  0.17894989, -0.07788992,
        -0.23207107,  0.68361866,  0.03119982,  0.10567835, -0.05552538,
        -0.03102371, -0.18795851, -0.28884593, -0.27810508, -0.1758991 ,
        -0.1246579 , -0.22765583, -0.28555155, -0.58930016, -0.8394351 ,
         0.20597254, -0.16140772, -0.19758859, -0.58260596, -0.39589792,
        -0.72690386, -0.89598405,  0.03571244]])

In [46]:
y_multi_pred

array([[ 1.1958351 ,  1.0042262 ,  1.0881205 ,  1.3191568 ,  0.4455724 ,
        -0.218968  ,  1.5589695 ,  1.1075264 ,  1.0349387 ,  0.7915236 ,
         0.99688077,  0.4442185 , -0.20829311,  1.2908653 ,  0.83135253,
         0.7313622 ,  0.5545079 ,  0.8354882 ,  0.62767106, -0.1741722 ,
         1.0410389 ,  0.54594785,  0.5876038 ,  0.3313658 ,  0.38258207,
         0.3509497 ,  0.01991402,  1.2071503 ,  0.29094827,  0.14206283,
         0.0408432 ,  0.32898492, -0.18829736, -0.46456954,  0.9014928 ,
         0.28740835,  0.08062498, -0.25775248,  0.1789498 , -0.0778899 ,
        -0.23207107,  0.68361866,  0.0312    ,  0.10567823, -0.05552532,
        -0.03102365, -0.18795848, -0.28884587, -0.27810514, -0.1758992 ,
        -0.12465773, -0.22765577, -0.28555155, -0.58930016, -0.83943516,
         0.2059726 , -0.16140789, -0.19758867, -0.5826063 , -0.3958981 ,
        -0.726904  , -0.8959843 ,  0.03571232]], dtype=float32)

In [47]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [48]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[-0.5843907594680786, -0.36296501755714417, -0.4808571934700012, -0.5399870276451111, -0.8982250690460205, -0.9459038972854614, -0.350142240524292]]]


In [49]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[-0.58439076, -0.36296502, -0.48085719, -0.53998703, -0.89822507,
        -0.9459039 , -0.35014224]])

In [50]:
y_future_pred

array([[-0.5843906 , -0.36296496, -0.4808569 , -0.5399871 , -0.8982251 ,
        -0.9459039 , -0.35014212]], dtype=float32)

In [51]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
